In [1]:
# !pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

import optuna

In [ ]:
from sklearn.metrics import roc_auc_score

# 데이터 로드

In [ ]:
import os

colab = os.path.isdir('./sample_data')
mount = os.path.isdir('/content/drive')
if colab:
    if not mount:
        from google.colab import drive
        drive.mount('/content/drive')
    base_path = '/content/drive/Othercomputers/내 컴퓨터/5_ML_Project/dulee/'
    data_path = '/content/drive/Othercomputers/내 컴퓨터/5_ML_Project/data/'
else:
    base_path = ''
    data_path = '../data/'

In [ ]:
submission_df = pd.read_csv(data_path+'sample_submission.csv', index_col='id')

train = pd.read_csv(base_path + 'train.csv', index_col='id')
test = pd.read_csv(base_path + 'test.csv', index_col='id')

##### 변수 설정

In [ ]:
X = train.drop(columns=['defects'])
y = train['defects']
X_test = test

# 모델 학습

### RandomForest

##### 기본 모델

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=61, shuffle=True, stratify=y)
# model = RandomForestClassifier(random_state=61)
# model.fit(X_train, y_train)

In [ ]:
# y_proba_randomforest_basic = model.predict_proba(X_val)[:, 1]
# roc_auc_score(y_val, y_proba_randomforest_basic)

##### hyper-parameter tuning

In [ ]:
import os
os.cpu_count()

8

In [ ]:
def optimizer(trial):

    n_estimators = trial.suggest_categorical('n_estimators', [100, 150, 200])  # 변경!! 500까지
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    max_depth = trial.suggest_int('max_depth', 11, 20)
    max_features = trial.suggest_float('max_features', 0.5, 0.8)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50)
    # min_impurity_decrease = trial.suggest_float('min_impurity_decrease', 0.0001, 0.001)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        min_samples_leaf=min_samples_leaf,
        max_depth=max_depth,
        max_features=max_features,
        min_samples_split=min_samples_split,
        # min_impurity_decrease=min_impurity_decrease,
        random_state=61,
        # n_jobs=-1,
        n_jobs=os.cpu_count()-1,
    )

    folds = StratifiedKFold() # RandomForest는 shuffle이 필요 없음.
    scores = []
    for train_idx, val_idx in folds.split(X, y):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])

        y_true = y.iloc[val_idx]
        y_proba = model.predict_proba(X.iloc[val_idx])[:, 1]
        scores.append(roc_auc_score(y_true, y_proba))
    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(optimizer, n_trials=100)

[I 2023-10-13 08:27:00,965] A new study created in memory with name: no-name-eed49c60-f01a-47e5-a746-97b23abbbd36
[I 2023-10-13 08:27:50,319] Trial 0 finished with value: 0.7913958556736717 and parameters: {'n_estimators': 150, 'min_samples_leaf': 9, 'max_features': 0.623060346039235, 'min_samples_split': 32}. Best is trial 0 with value: 0.7913958556736717.
[I 2023-10-13 08:28:23,662] Trial 1 finished with value: 0.7910613231096709 and parameters: {'n_estimators': 100, 'min_samples_leaf': 1, 'max_features': 0.6322167212783414, 'min_samples_split': 37}. Best is trial 0 with value: 0.7913958556736717.
[I 2023-10-13 08:29:03,258] Trial 2 finished with value: 0.7912996432715177 and parameters: {'n_estimators': 100, 'min_samples_leaf': 15, 'max_features': 0.77146303593656, 'min_samples_split': 15}. Best is trial 0 with value: 0.7913958556736717.
[I 2023-10-13 08:29:45,067] Trial 3 finished with value: 0.7912818272155407 and parameters: {'n_estimators': 150, 'min_samples_leaf': 3, 'max_featu

In [ ]:
print("Best roc_auc_score: %.4f" % study.best_value)
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

Best roc_auc_score: 0.7916
Best params:  {'n_estimators': 200, 'min_samples_leaf': 12, 'max_features': 0.6014686638877126, 'min_samples_split': 24}


##### best model

In [ ]:
model_best = RandomForestClassifier(**study.best_trial.params, random_state=61)
model_best.fit(X, y)

RandomForestClassifier(max_features=0.6014686638877126, min_samples_leaf=12,
                       min_samples_split=24, n_estimators=200, random_state=61)

In [ ]:
y_proba = model_best.predict_proba(X_test)[:, 1]
y_proba

array([0.25851054, 0.14009606, 0.62245951, ..., 0.16399582, 0.10925127,
       0.90157781])

##### 분석

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_max_features,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.791396,2023-10-13 08:27:00.967904,2023-10-13 08:27:50.318899,0 days 00:00:49.350995,0.623060,9,32,150,COMPLETE
1,1,0.791061,2023-10-13 08:27:50.320856,2023-10-13 08:28:23.661848,0 days 00:00:33.340992,0.632217,1,37,100,COMPLETE
2,2,0.791300,2023-10-13 08:28:23.663686,2023-10-13 08:29:03.257667,0 days 00:00:39.593981,0.771463,15,15,100,COMPLETE
3,3,0.791282,2023-10-13 08:29:03.259769,2023-10-13 08:29:45.067203,0 days 00:00:41.807434,0.550841,3,6,150,COMPLETE
4,4,0.791148,2023-10-13 08:29:45.069178,2023-10-13 08:30:21.724783,0 days 00:00:36.655605,0.724363,8,41,100,COMPLETE
...,...,...,...,...,...,...,...,...,...,...
95,95,0.791586,2023-10-13 09:48:31.074832,2023-10-13 09:49:26.430232,0 days 00:00:55.355400,0.604497,12,22,200,COMPLETE
96,96,0.791586,2023-10-13 09:49:26.432461,2023-10-13 09:50:21.928075,0 days 00:00:55.495614,0.609237,12,23,200,COMPLETE
97,97,0.791586,2023-10-13 09:50:21.929925,2023-10-13 09:51:17.285942,0 days 00:00:55.356017,0.609631,12,23,200,COMPLETE
98,98,0.791586,2023-10-13 09:51:17.287735,2023-10-13 09:52:12.966721,0 days 00:00:55.678986,0.612970,12,22,200,COMPLETE


In [ ]:
study.trials_dataframe().sum()

<ipython-input-47-655b4ce5e784>:1: FutureWarning:

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



number                                                                   4950
value                                                               79.145498
duration                                               0 days 01:26:07.320176
params_max_features                                                 59.416866
params_min_samples_leaf                                                  1425
params_min_samples_split                                                 2276
params_n_estimators                                                     18400
state                       COMPLETECOMPLETECOMPLETECOMPLETECOMPLETECOMPLE...
dtype: object

In [ ]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

# 제출

In [ ]:
submission_df['defects'] = y_proba
submission_df.to_csv(base_path + 'submission_randomforest.csv')
submission_df

,defects
id,
101763,0.258511
101764,0.140096
101765,0.622460
101766,0.479302
101767,0.178362
...,...
169600,0.209872
169601,0.118576
169602,0.163996


In [ ]:
base_path

'/content/drive/Othercomputers/내 컴퓨터/5_ML_Project/dulee/'